In [1]:
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import numpy as np
from keras.preprocessing import image
import os

Using TensorFlow backend.


In [2]:
base_model = ResNet50(weights='imagenet', include_top=False)

In [3]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, None, None, 6 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [7]:
for l in base_model.layers[:3]:
    print(l.name)
    print(l.get_config())

input_1
{'batch_input_shape': (None, None, None, 3), 'dtype': 'float32', 'sparse': False, 'name': 'input_1'}
conv1_pad
{'name': 'conv1_pad', 'trainable': True, 'dtype': 'float32', 'padding': ((3, 3), (3, 3)), 'data_format': 'channels_last'}
conv1_conv
{'name': 'conv1_conv', 'trainable': True, 'dtype': 'float32', 'filters': 64, 'kernel_size': (7, 7), 'strides': (2, 2), 'padding': 'valid', 'data_format': 'channels_last', 'dilation_rate': (1, 1), 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'GlorotUniform', 'config': {'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}


In [9]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1000, activation='softmax')(x)

In [10]:
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [12]:
train_dir = os.environ['IMAGE_TRAIN_INPUT_PATH']
train_datagen = image.ImageDataGenerator()
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=50,
    class_mode='binary')

Using TensorFlow backend.


In [13]:
model.fit_generator(train_generator, steps_per_epoch=10, epochs=100)

In [15]:
MODEL_PATH = os.environ['MODEL_INFERENCE_PATH']
print(MODEL_PATH)
tf.keras.experimental.export_saved_model(model, MODEL_PATH + '/SavedModel')

Found 3 images belonging to 2 classes.
